In [24]:
import numpy as np
from nltk import edit_distance
import pandas as pd
from sklearn.cluster import HDBSCAN

1. take dataset 
2. take current row 
3. split it into: 
    - CDR3, V, J 
    - Epitope, MHC A, MHC B, MHC class 
4. take next (random) row (or for each)
5. split it into: 
    - CDR3, V, J 
    - Epitope, MHC A, MHC B, MHC class
6. compute levenshtein distance for: 
    - CDR3 region (row A to row B)
    - Epitopes (row A to row B)
7. if they are NOT to similar => search new row B 

In [25]:
def levenshtein_metric(x, y, df, column_name_A, column_name_B):
    i, j = int(x[0]), int(y[0]) # get indices
    return edit_distance(df.iloc[i][column_name_A], df.iloc[j][column_name_B])

In [26]:
read_path_beta = "../data/customDatasets/beta_concatenated.tsv"
beta_df = pd.read_csv(read_path_beta, sep="\t")

/tmp/ipykernel_90080/624418882.py:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  beta_df = pd.read_csv(read_path_beta, sep="\t")


In [27]:
beta_df

,TCR_name,TRAV,TRAJ,TRA_CDR3,TRBV,TRBJ,TRB_CDR3,TRAC,TRBC,TRA_leader,...,TRA_5_prime_seq,TRA_3_prime_seq,TRB_5_prime_seq,TRB_3_prime_seq,Epitope,Score_TRA,Score_TRB,MHC A,MHC B,MHC class
0,1,NaN,NaN,NaN,TRBV13*01,TRBJ1-5*01,CASSYLPGQGDHYSNQPQHF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLKEKGGL,NaN,2.0,HLA-B*08,B2M,MHCI
1,0,NaN,NaN,NaN,TRBV13*01,TRBJ1-5*01,CASSFEAGQGFFSNQPQHF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLKEKGGL,NaN,2.0,HLA-B*08,B2M,MHCI
2,2,NaN,NaN,NaN,TRBV13*01,TRBJ1-5*01,CASSFEPGQGFYSNQPQHF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLKEKGGL,NaN,2.0,HLA-B*08,B2M,MHCI
3,3,NaN,NaN,NaN,TRBV13*01,TRBJ1-5*01,CASSYEPGQVSHYSNQPQHF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLKEKGGL,NaN,2.0,HLA-B*08,B2M,MHCI
4,4,NaN,NaN,NaN,TRBV14*01,TRBJ2-1*01,CASSALASLNEQFF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLKEKGGL,NaN,2.0,HLA-B*08,B2M,MHCI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53306,13697,TRAV25,TRAJ28,CAGSGAGSYQLTF,NaN,NaN,CASSLEGQASSYEQYF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,CLGGLLTMV,NaN,NaN,NaN,NaN,NaN
53307,13698,TRAV25,TRAJ28,CAGLGAGSYQLTF,NaN,NaN,CASSLEGQGASYEQYF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,CLGGLLTMV,NaN,NaN,NaN,NaN,NaN
53308,13699,TRAV17*01,TRAJ11*01,CATEGNSGYSTLTF,NaN,NaN,CASSSQGGNYGYTF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLYALALLL,NaN,NaN,NaN,NaN,NaN
53309,13700,TRAV17*01,TRAJ11*01,CATEGDSGYSTLTF,NaN,NaN,CASSYQGGNYGYTF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,FLYALALLL,NaN,NaN,NaN,NaN,NaN


In [28]:
max_index = len(beta_df) - 1 
negative_epitopes = []

In [29]:
def search_negative_epitope(df, index):
    df = df
    index = index
    epitope = df["Epitope"][index]
    print(epitope)
    random_epitope_index = np.random.randint(0, max_index)
    random_epitope = df["Epitope"][random_epitope_index]
    leven_dist = edit_distance(epitope, random_epitope)

    if(leven_dist >= 5): 
        negative_epitopes.append(random_epitope)
    else: 
        search_negative_epitope(df, index)

In [30]:
for i, epitope in enumerate(beta_df["Epitope"]): 
    search_negative_epitope(beta_df, i)

FLKEKGGL
FLKEKGGL
FLKEKGGL
FLKEKGGL
FLKEKGGL
FLKEQGGL
FLKEQGGL
FLKEQGGL
FLKEQGGL
FLKETGGL
FLKETGGL
FLKETGGL
FLKETGGL
FLKEMGGL
FLKEMGGL
FLKEMGGL
FLKEMGGL
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LQPFPQPELPYPQPQ
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL
LLQTGIHVRVSQPSL

In [31]:
negative_epitopes_df = pd.DataFrame(negative_epitopes, columns=["Negative Epitope"])
negative_epitopes_df

,Negative Epitope
0,FLYALALLL
1,TTDPSFLGRY
2,RAKFKQLL
3,KLWAQCVQL
4,LPRRSGAAGA
...,...
53306,GILGFVFTL
53307,RVAGDSGFAAY
53308,KLGGALQAK
53309,WLMWLIINL
